In [1]:
# ! ssh -fNL 27017:localhost:27017 bonnen@rxdhawkins.me

In [1]:
import datetime, os, json, boto, pymongo, numpy, h5py
import numpy as np
from sklearn.metrics import confusion_matrix
from boto.mturk.connection import MTurkConnection
a = np.array

In [6]:
live = 1
mturk_platform = 'live'
credential_location = '../credentials/'
rootkey = 'auth.json'

with open(credential_location + rootkey) as f: 
    key_info = json.load(f)
    
# bonus subject
host = 'mechanicalturk.amazonaws.com'
# extract account info and define platform

if live: host = 'mechanicalturk.amazonaws.com'
else: host = 'mechanicalturk.sandbox.amazonaws.com'

# establish mturk connection
mturk = MTurkConnection(aws_access_key_id=key_info['access_id'],
                        aws_secret_access_key=key_info['secret_key'],
                        host=host)

In [9]:
# extract hit info from submission inventory
submission_file = 'HIT_submission_records_%s.npy'%(mturk_platform)
submission_records = np.load(submission_file).item()
submissions = submission_records.keys()

In [15]:
mongo_auth = "/Users/biota/memory/sleep_affect_memory/experiment/.credentials/auth.json"
access_info = json.load(open(mongo_auth))

# open tunnel -- requires established connection [ssh -fNL 27017:localhost:27017 bonnen@rxdhawkins.m]
mongo_tunnel = 'mongodb://' + access_info['user'] + ':' + access_info['password'] + '@127.0.0.1'

# conect, define collection and database
connection = pymongo.MongoClient(mongo_tunnel)
data_base = connection['sleep_affect_memory']
collection = data_base['replication']

In [90]:

def get_hit_and_worker_ids(i_submission): 

    submission_info = submission_records[i_submission]
    hit_id = submission_info['hit_id']
    subject_ids = a(collection.find({'hit_id':hit_id}).distinct('worker_id'))
    
    return hit_id, subject_ids

def generate_record_for_inventory(subject_ids, submission_records, i_submission): 
    
    if len(subject_ids): 
        
        try: 
            submission_records[i_submission]['subject_info'] 
        except: 
            submission_records[i_submission]['subject_info'] = {} 
            #print('creating new submission inventory... ')

        for i_subject in subject_ids: 

            try: 
                submission_records[i_submission]['subject_info'][i_subject] 
            except: 
                submission_records[i_submission]['subject_info'][i_subject] = {} 
                #print('creating new subject inventory')
    else: 
        
        print('\n no subject data \n')
        
    return submission_records 

def bonus_protocol(hit_id, i_submission,manual=True, manual_rebonus=0): 
    
    
    w_data = {i['worker_id']: i for i in collection.find({'hit_id':hit_id, "data_type":"feedback"})}  
    
    for i_subject in list(w_data.keys()): 

        assignment_id = w_data[i_subject]['assignment_id']
        bonus_amount =  float('%.2f'%float(w_data[i_subject]['trial_data']['total_bonus_earned']))
        converted_bonus = boto.mturk.price.Price(bonus_amount) 

        try: 
            submission_records[i_submission]['subject_info'][i_subject]
            if submission_records[i_submission]['subject_info'][i_subject]['assigned_bonus'] == 1 : 
                needs_bonus = 0 
            else: 
                needs_bonus = 1 
        except: 
            needs_bonus = 1  

        converted_bonus = performance_bonus = boto.mturk.price.Price(bonus_amount)
        if manual_rebonus == 1: 
            needs_bonus = 1
            
        if needs_bonus: 

            if manual:
                
                try: 
                    w_data[i_subject]['trial_data']['worker_feedback']
                    print('bonus amount: %f'%bonus_amount)
                except: 
                    print("no feedback data for subject")
                
                MSG = input("Enter a custom message? Pressing enter reverts to default. ")
                if len(MSG) == 0: MSG = "Thanks for participating in our study!"
                BONUS = input("Amount to bonus worker? Pressing enter reverts to {} ".format(bonus_amount))
                if len(BONUS) == 0: BONUS = bonus_amount
                converted_bonus = boto.mturk.price.Price(BONUS) 
                
            else: 
                
                MSG = "Thanks for participating in our study!"
                BONUS = bonus_amount
                converted_bonus = boto.mturk.price.Price(BONUS) 

            try: 
                print(i_subject, assignment_id,  converted_bonus)
                s = mturk.grant_bonus(i_subject, assignment_id,  converted_bonus, MSG);
                print('success!\n',)
                submission_records[i_submission]['subject_info'][i_subject]['assigned_bonus'] = 1
                submission_records[i_submission]['subject_info'][i_subject]['bonus_amount'] = converted_bonus
                submission_records[i_submission]['subject_info'][i_subject]['return_message']  = {type(s): s}

            except Exception as e:

                ERROR = {type(e):e}
                submission_records[i_submission]['subject_info'][i_subject]['assigned_bonus'] = 0
                submission_records[i_submission]['subject_info'][i_subject]['bonus_amount'] = 0 
                submission_records[i_submission]['subject_info'][i_subject]['return_message']  = {type(e):e}
                print('ERROR:', type(e),)

            np.save(submission_file, submission_records)

        else: 

            print('already bonused %s'%i_subject)
            
    return submission_records


In [96]:

for i_submission in submission_records.keys(): 
    
    print('\n%s\n'%i_submission)

    hit_id, subject_ids = get_hit_and_worker_ids(i_submission)
    submission_records = generate_record_for_inventory(subject_ids, submission_records, i_submission) 
    submission_records = bonus_protocol(hit_id, i_submission, manual=True) 



submission_0

already bonused A2CUBA783W1689
already bonused AWAW665TQQP2F
